# <span style='color:Blue'> Retrieval Augmentation Generation with SingleStoreDB

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Building a Q+A LangChain App with SingleStoreDB</h1>
    </div>
</div>

### <span style='color:Blue'>   Using Langchain with  SingleStoreDB & Open AI with Siemens Balance Sheet

### 1.Install and import dependencies

In [7]:
!pip install langchain --quiet
!pip install --upgrade openai==0.28.1 --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install --upgrade unstructured==0.10.14 --quiet

#### Data Source Link:https://assets.new.siemens.com/siemens/assets/api/uuid:9dce298f-ba1a-496c-b062-75dccdd45bed/annual-report-2022.pdf

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Using the <b>Edit Firewall</b> button in the top right, add the following to the SingleStoreDB Cloud notebook firewall:
            <ul style="list-style: none;">
                <li><b>assets.new.siemens.com</b></li>
            </ul>
        </p>
    </div>
</div>

### 2.Import the libraries

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os

### 3.Load knowledge base document-Siemens Annual Report -2021-2022.

In [12]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://assets.new.siemens.com/siemens/assets/api/uuid:9dce298f-ba1a-496c-b062-75dccdd45bed/annual-report-2022.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


#### 4. Using LangChain split the document into chunks.

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 113338 characters in your document


In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 58 pages


### <span style='color:Green'> 5.Using Open AI generate the embeddings for the split chunks and store in SingleStoreDB table

In [16]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


### <span style='color:Green'>5.1 Create the embeddings using Open AI and store into the table pdf_demo.

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SingleStoreDB to make connections to the selected database.</p>
    </div>
</div>

In [25]:
%%sql
#Create the table pdf_demo

DROP TABLE IF EXISTS pdf_demo;
CREATE TABLE IF NOT EXISTS pdf_demo (
    id INT PRIMARY KEY,
    content TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    vector BLOB
);

++
||
++
++

In [253]:
#create the embeddings using Open AI and store into the table pdf_demo.
import json
import sqlalchemy as sa
from langchain.embeddings import OpenAIEmbeddings
from singlestoredb import create_engine

conn = create_engine().connect()

embedder = OpenAIEmbeddings()

# Fetch all embeddings in one call
embeddings = embedder.embed_documents([doc.page_content for doc in texts])

# Build query parameters
params = []
for i, (text_content, embedding) in enumerate(zip(texts, embeddings)):
    params.append(dict(id=i+1, content=text_content, vector=json.dumps(embedding)))

stmt = sa.text("""
    INSERT INTO pdf_demo (
        id,
        content,
        vector
    )
    VALUES (
        :id,
        :content,
        JSON_ARRAY_PACK_F32(:vector)
    )
""")

conn.execute(stmt, params)

## <span style='color:Green'>5.1: Retrieve using Semantic search

### Prompt:What is the dividend for  the year 2022?

In [254]:
query_text = "What is the dividend for  the year 2022?"

query_embedding = embedder.embed_documents([query_text])[0]

stmt = sa.text("""
    SELECT
        content,
        DOT_PRODUCT_F32(JSON_ARRAY_PACK_F32(:embedding), vector) AS score
    FROM pdf_demo
    ORDER BY score DESC
    LIMIT 1
""")

results = conn.execute(stmt, dict(embedding=json.dumps(query_embedding)))

for row in results:
    print(row[0])

page_content="Of profit for the year totaling DKK 59 million, DKK 58 million is expected to be\n\ndistributed as a dividend to the parent company, Siemens International Holding\n\nB.V., Den Haag, the Netherlands.\n\nThe average number of employees decreased from 492 in the financial year\n\n2020/2021 to 480 in the financial year 2021/2022.\n\nIn the financial year 2021/2022, Siemens A/S did not apply for financial support\n\nfrom the COVID-19 aid packages, e.g., the wage compensation scheme or the\n\ncompensation scheme for fixed costs.\n\nConsiderable uncertainty reigns due to the current geopolitical situation, and there\n\nis a risk of a global financial recession, which will have a negative impact on the\n\nDanish economy. Siemens A/S expects an increase in revenue of 7-11% in 2022/2023\n\nbased on the solid order book, which has been built up during 2021/2022. However,\n\nthe risk of a general financial downturn is high, and the estimate is thus subject to\n\nsome uncertainty.\n\n

### <span style='color:Green'>5.1: Generate /Augment using Open AI

In [255]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {row[0]}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

According to the given document, the dividend for the year 2022 is expected to be DKK 58 million.


## <span style='color:Blue'>  5.2 LangChain and SingleStoreDB integration https://python.langchain.com/docs/integrations/vectorstores/singlestoredb

### <span style='color:Blue'> 5.2 Create and load the embeddings into a new table  with fields(content,vector,metadata).


In [20]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [22]:
#import langchain.vectorstores.singlestoredb as s2
from langchain.vectorstores import SingleStoreDB
#from langchain.vectorstores.utils import DistanceStrategy

#s2.ORDERING_DIRECTIVE["DOT_PRODUCT"] = s2.ORDERING_DIRECTIVE[DistanceStrategy.DOT_PRODUCT]

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "pdf_lab2",
    #distance_strategy = "DOT_PRODUCT",
)

In [ ]:
%%sql
%%sql
select * from pdf_lab2;

###   <span style='color:Blue'>  5.2 Based on the user question, retrieves  the top k closest content and vectors using similarity search (question,content)

In [262]:
query = "What is the dividend for  the year 2022?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Of profit for the year totaling DKK 59 million, DKK 58 million is expected to be

distributed as a dividend to the parent company, Siemens International Holding

B.V., Den Haag, the Netherlands.

The average number of employees decreased from 492 in the financial year

2020/2021 to 480 in the financial year 2021/2022.

In the financial year 2021/2022, Siemens A/S did not apply for financial support

from the COVID-19 aid packages, e.g., the wage compensation scheme or the

compensation scheme for fixed costs.

Considerable uncertainty reigns due to the current geopolitical situation, and there

is a risk of a global financial recession, which will have a negative impact on the

Danish economy. Siemens A/S expects an increase in revenue of 7-11% in 2022/2023

based on the solid order book, which has been built up during 2021/2022. However,

the risk of a general financial downturn is high, and the estimate is thus subject to

some uncertainty.

The Company's profit from ordinary activit

###  <span style='color:Blue'>  5.2  Augmented response to the users question

In [263]:
import openai

prompt = f"The user asked: {query}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

Based on the information provided, the dividend for the year 2022 is expected to be DKK 58 million.


###   5.3 When the knowledge base  is not given ,LLM answers as below 

In [281]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.8)


In [283]:
llm.predict("What is the dividend for  the year 2022 in Siemens?")

'\n\nIt is not possible to provide an accurate answer to this question as the dividend for 2022 has not yet been determined by Siemens.'

## Clean up

In [ ]:
%%sql
DROP DATABASE IF EXISTS pdf_db;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>

==========================================*****====================================

### 5.4 Converse with Chatbot

In [146]:
from langchain.chains import ConversationalRetrievalChain
#from langchain.text_splitter import CharacterTextSplitter
#from langchain.llms import OpenAI
#import langchain.vectorstores.singlestoredb as s2
#from langchain.vectorstores import SingleStoreDB

In [317]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [324]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(llm=OpenAI(temperature=0.7), memory_key="chat_history", output_key='answer', return_messages=True)

### Create a ConversationalRetrievalChain

In [ ]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm=OpenAI(temperature=0.7), 
    retriever=docsearch.as_retriever(), memory=memory)

# Create the chain
qa = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(temperature=0.7), 
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
        memory_key="chat_history",
        return_messages=True,
)

In [ ]:
# Initialize chat history list
chat_history = []

### Begin conversing!

In [ ]:
query = "What is the dividend for  the year 2022 in Siemens?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

In [336]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [ ]:
# Notice that the Chatbot knows that "two years before [that]" is "two years before [2022]"
query = "What was it two years before that?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

In [ ]:
# Add the answer to the chat history
chat_history.append((query, result["answer"]))

In [43]:
from langchain.prompts import PromptTemplate
prompt_template = """
Use the following pieces of context to answer the question at the end. If
you're not sure, just say so. If there are potential multiple answers,
summarize them as possible answers.
{context}
Question: {question}
Answer:
"""

PROMPT = PromptTemplate(template=prompt_template,
input_variables=["context", "question"])

# Initialize RetrievalQA object
qa_chain = load_qa_chain(OpenAI(), chain_type="stuff")
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=OpenAI(model_name='gpt-4-0613'),
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs)

# Query the data
query = "What is the dividend for  the year 2022?"
qa.run(query)

NameError: name 'load_qa_chain' is not defined

## Clean up

In [ ]:
%%sql
DROP DATABASE IF EXISTS pdf_db;

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>